# Install python dependencies

In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas scikit-learn Keras tensorflow

In [ ]:
pip install tensorflow==2.2.0

# Imports dependencies.

In [3]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

/home/frankjony17/Projects/venvs/ia-course/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Load datasets -> Dados e Clases

In [4]:
data_entry = pd.read_csv("dataset/entry_breast.csv")
data_class = pd.read_csv("dataset/exits_breast.csv")

# Configures the model for training

In [9]:
def create_rede():
    _classifier = Sequential()  # Sequential model.
    # Connected NN layer (Input Layer)
    _classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
    # Dropout
    _classifier.add(Dropout(0.2))
    # Hidden Layer
    _classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
    # Dropout
    _classifier.add(Dropout(0.2))
    # Hidden Layer
    _classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
    # Output Layer
    _classifier.add(Dense(units=1, activation='sigmoid'))
    # 
#     adam = Adam(lr=0.001, decay=0.0001, clipvalue=0.5)
    # Configures the model for training
    _classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return _classifier

# Implementation of the scikit-learn classifier API for Keras

In [10]:
classifier = KerasClassifier(build_fn=create_rede, epochs=120, batch_size=20)

In [ ]:
# cv -> cross-validation
result = cross_val_score(estimator=classifier, X=data_entry, y=data_class, cv=10, scoring='accuracy')

In [16]:
media = result.mean()
standard_deviation = result.std()

In [17]:
print(media)
print(standard_deviation)

0.894454887218045
0.06906746646732673


In [18]:
print(result)

[0.73684211 0.87719298 0.94736842 0.84210526 0.96491228 0.94736842
 0.89473684 0.96491228 0.92982456 0.83928571]


In [ ]:
[0.78947368 0.87719298 0.89473684 0.8245614  0.87719298 0.89473684 0.92982456 0.85964912 0.87719298 0.96428571] # widtout Dropout
[0.75438596 0.9122807  0.98245614 0.89473684 0.94736842 0.94736842 0.96491228 0.94736842 0.89473684 0.96428571] # widt Dropout

[0.8245614  0.9122807  0.94736842 0.85964912 0.92982456 0.96491228 0.96491228 0.96491228 0.92982456 0.80357143] 0.91 0.05
[0.8245614  0.9122807  0.94736842 0.84210526 0.89473684 0.96491228 0.92982456 0.96491228 0.89473684 0.96428571] 0.91 0.04